In [2]:
import json
import glob
from tld import get_tld

In [27]:
EMAIL_PREFIX = "timthesis40+"
EMAIL_SUFFIX =  "@gmail.com"
OG_FIELD_INPUTS = {'tel': "497413093", 'organization': "Sample Company", 'street-address': "Rue Jean Lorette 169", 'address-line1': "number 89",
                'address-line2': "box 2", 'address-line3': "Thuin", 'address-level2': "Thuin", 'address-level1': "Thuin", 'postal-code': "6530",
                'country': "Belgium", 'cc-name': "Harold Gonzalez", 'name': "Harold", 'given-name': "Harry", 'additional-name': "Jerry",
                'family-name': "Gonzalez", 'cc-number': "4929846523784508", 'cc-exp-month': "02", 'cc-exp-year': "25", 'cc-exp': "02/25",
                'cc-type': "Mastercard", 'username': "cosicadam", 'password': "myPwd1111111111111=", 'default': "Adam"}
FIELD_INPUTS_NO_NUMBERS = {'tel': "497413093", 'organization': "Sample Company", 'street-address': "Rue Jean Lorette 169", 'address-line1': "number 89",
                'address-line2': "box 2", 'address-line3': "Thuin", 'address-level2': "Thuin", 'address-level1': "Thuin", 'country': "Belgium",
                'cc-name': "Harold Gonzalez", 'name': "Harold", 'given-name': "Harry", 'additional-name': "Jerry", 'family-name': "Gonzalez",
                'cc-number': "4929846523784508", 'cc-type': "Mastercard", 'username': "cosicadam", 'password': "myPwd1111111111111=", 'default': "Adam"}

REQUEST_TRESHHOLD = 1

In [28]:
OUTPUT_MAPPING = {0: "Not present", 1: "No single compbination", 2: "Combined presence, no input", 3: "Combined presence, input", 4: "Single domain, no input", 5: "Single domain, input"}

In [29]:
def StatusEventListeners(eventListenersResult):
    if not eventListenersResult["pageContainsAllEvents"]:
        # Page doesn't have required event listeners, likelyhood of prescense low
        return 0, [], []
    
    if len(eventListenersResult["Domains"]) == 0:
        # The page has the eventlisteners spread over all domains, not likely this would be the case
        return 1, [], []
    
    scriptsContainingAll = []
    domains = list(eventListenersResult["Domains"].keys())

    for domain in eventListenersResult["Domains"]:
        scriptsContainingAll += eventListenersResult["Domains"][domain]["scriptsContainingAllEvents"]

    if len(scriptsContainingAll) == 0:
        # No single script has all listeners, srs possible, but less likely then if one script had all listeners
        return 2, domains, []
    
    return 3, domains, scriptsContainingAll

In [30]:
def StatusAPI(apiResult):
    if len(apiResult) == 0:
        # No fields filled
        return -1, [], []
    
    domains = set()
    urls = set()
    result = 0

    for domain in apiResult:
        for url in apiResult[domain]:
            if apiResult[domain][url][1] > result:
                result = apiResult[domain][url][1]
            if apiResult[domain][url][1] >= 2:
                domains.add(domain)
                urls.add(url)

    # if len(domains) != 0:
    #     return result, list(domains), list(urls)
    
    # for domain in apiResult:
    #     for url in apiResult[domain]:
    #         if apiResult[domain][url][1] > result:
    #             result = apiResult[domain][url][1]
    #         if apiResult[domain][url][1] == 1:
    #             domains.add(domain)
    #             urls.add(url)
    
    return result, list(domains), list(urls)

In [31]:
def StatusRequests(requestResult, eventDomains, eventScripts):
    if len(requestResult) == 0:
        return 0, []
    
    domainMatched = []
    scriptMatched = []

    for initiatorResult in requestResult:
        if initiatorResult["count"] < REQUEST_TRESHHOLD:
            continue

        for domain in eventDomains:
            # Check if a domain overlap exists between requests and events
            if domain in initiatorResult["domains"]:
                for script in eventScripts:
                    # Check if a scrips overlap exists between requests and events
                    if script in initiatorResult["urls"]:
                        scriptMatched.append(initiatorResult)
    
    if len(scriptMatched) == 0:
        return 1, []
    
    return 2, scriptMatched

In [40]:
def CombineResults(eventStatus, eventDomains, eventScripts, apiStatus, apiDomains, apiScripts, requestStatus, requestScripts):

    if requestStatus == 1 and eventStatus > 1:
            # Requests found, but no match
            return 1, [], []
    
    # Match in script between event listeners and requests

    # Inputs were filled
    if apiStatus != -1:
        scripts = set()
        domains = set()
        result = 3
        # probable script the last in initiator chain?
        for initiatorResult in requestScripts:
            script = initiatorResult["urls"][-1]
            if script == "__puppeteer_evaluation_script__":
                if len(initiatorResult["urls"]) > 1:
                    script = initiatorResult["urls"][-2]  

            scripts.add(script)
            res = get_tld(script, as_object=True, fail_silently=True)
            if res is not None:
                domains.add(res.domain)
                # Check if final sending domain is the one with the events
                if res.domain in eventDomains and res.domain in apiDomains:
                    result = 5

        return result, list(domains), list(scripts)

    # No inputs were filled
    else:
        scripts = set()
        domains = set()
        result = 2
        # probable script the last in initiator chain?
        for initiatorResult in requestScripts:
            script = initiatorResult["urls"][-1]
            if script == "__puppeteer_evaluation_script__":
                if len(initiatorResult["urls"]) > 1:
                    script = initiatorResult["urls"][-2]            

            scripts.add(script)
            res = get_tld(script, as_object=True, fail_silently=True)
            if res is not None:
                domains.add(res.domain)
                # Check if final sending domain is the one with the events
                if res.domain in eventDomains:
                     result = 4

        return result, list(domains), list(scripts)

In [33]:
# Return likelyhood, identified domains, scripts and the eventlisteners status
def CheckSRSLikelyhood(siteData):
    eventStatus, eventDomains, eventScripts = StatusEventListeners(siteData["EventListenerResults"])
    print(eventStatus, eventDomains, eventScripts)
    if eventStatus == 0:
        return 0, [], [], 0
    
    requestStatus, requestScripts = StatusRequests(siteData["RequestResults"], eventDomains, eventScripts)
    print(requestStatus, requestScripts)
    if requestStatus == 0:
        return 0, [], [], eventStatus

    apiStatus, apiDomains, apiScripts = StatusAPI(siteData["APIResults"])
    print(apiStatus, apiDomains, apiScripts)

    result, domains, scripts = CombineResults(eventStatus, eventDomains, eventScripts, apiStatus, apiDomains, apiScripts, requestStatus, requestScripts)
    return result, domains, scripts, eventStatus

In [34]:
filePath = "SortedOutput2/wired.com.json"

In [41]:
siteName = filePath[0:filePath.rfind(".")]
siteName = siteName[siteName.find("/")+1:]
print(siteName)

siteDomain = None
res = get_tld(siteName, as_object=True, fail_silently=True)
print(res)
if res is not None:
    siteDomain = res.domain

print(siteDomain)

# Load the data from the .json file
f = open(filePath, encoding="utf-8") 
allData = json.load(f)
f.close()

result, domains, scripts, eventStatus = CheckSRSLikelyhood(allData)

print({"result": result, "domains": domains, "scripts": scripts, "eventStatus": eventStatus})

3 ['condenastdigital', 'hotjar', 'imasdk', 'parsely'] ['https://pixel.condenastdigital.com/sparrow.min.js', 'https://script.hotjar.com/modules.15845db2f7d4b3e39af3.js', 'https://imasdk.googleapis.com/js/sdkloader/ima3.js', 'https://cdn.parsely.com/keys/wired.com/p.js']
2 [{'count': 2, 'domains': ['condenastdigital', 'wired'], 'inputsPresent': [], 'urls': ['https://pixel.condenastdigital.com/sparrow.min.js', 'https://www.wired.com/verso/static/9738.c3d147421dc2d899a182.js', 'https://www.wired.com/verso/static/9742.8be795300da626b96b3c.js']}, {'count': 13, 'domains': ['condenastdigital'], 'inputsPresent': [], 'urls': ['https://pixel.condenastdigital.com/sparrow.min.js']}, {'count': 1, 'domains': ['imasdk', 'd2c8v52ll5s99u', 'cookielaw'], 'inputsPresent': [], 'urls': ['https://imasdk.googleapis.com/js/sdkloader/ima3.js', 'https://d2c8v52ll5s99u.cloudfront.net/player/main-122ba8edff12a051f727.js', 'https://cdn.cookielaw.org/scripttemplates/202211.2.0/otTCF.js', 'https://cdn.cookielaw.org/s

In [16]:
# siteName = "https://vintageking.com"
siteName = "https://000cheapdomains.com"

siteDomain = None
res = get_tld(siteName, as_object=True, fail_silently=True)
print(res)
if res is not None:
    siteDomain = res.domain

print(siteDomain)

com
000cheapdomains
